In [1]:
"""
!wget -q https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
!unzip -o wwm_uncased_L-24_H-1024_A-16.zip
"""    

'\n!wget -q https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip\n!unzip -o wwm_uncased_L-24_H-1024_A-16.zip\n'

In [36]:
import tensorflow as tf
import keras
from keras_radam import RAdam
from keras import backend as K
from keras_bert import load_trained_model_from_checkpoint, Tokenizer

import pandas as pd

from tqdm import tqdm
import codecs
import os

import socket

In [ ]:
dataset = "science"

currentDir = os.path.dirname(os.path.realpath("__file__"))
bertDir = os.path.join(currentDir, "..", "wwm_cased_L-24_H-1024_A-16")
vocabDir = os.path.join(bertDir, "vocab.txt")
driver_ip = socket.gethostbyname(socket.gethostname())
spark_conf = [('spark.kubernetes.authenticate.caCertFile', '/var/run/secrets/kubernetes.io/serviceaccount/ca.crt'), \
                                   ('spark.kubernetes.authenticate.oauthTokenFile','/var/run/secrets/kubernetes.io/serviceaccount/token'), \
                                   ('spark.kubernetes.authenticate.driver.serviceAccountName','spark-driver-sa'), \
                                   ('spark.kubernetes.namespace','spark'), \
                                   ('spark.driver.pod.name','spark-driver'), \
                                   ('spark.executor.instances','16'), \
                                   ('spark.kubernetes.container.image','gcr.io/sarcasm-3wx3ce6drvftuy/spark-v2.4.4-worker:latest'), \
                                   ('spark.driver.host','spark-driver.spark.svc.cluster.local'), \
                                   ('spark.driver.port','29413'), \
                                   ('spark.driver.bindAddress',driver_ip), \
                                   ('spark.executor.memory','6700m'), \
                                   ('spark.executor.cores','1'), \
                                   ('spark.kubernetes.driverEnv.GCS_PROJECT_ID', 'sarcasm-3wx3ce6drvftuy'), \
                                   ('spark.kubernetes.driverEnv.GOOGLE_APPLICATION_CREDENTIALS', '/mnt/secrets/sarc-bucket-sa.json'), \
                                   ('spark.kubernetes.driver.secrets.sarc-bucket-sa','/mnt/secrets'), \
                                   ('spark.kubernetes.executor.secrets.sarc-bucket-sa','/mnt/secrets'), \
                                   ('spark.executorEnv.GCS_PROJECT_ID','sarcasm-3wx3ce6drvftuy'), \
                                   ('spark.executorEnv.GOOGLE_APPLICATION_CREDENTIALS','/mnt/secrets/sarc-bucket-sa.json'), \
                                   ('spark.hadoop.google.cloud.auth.service.account.enable','true'), \
                                   ('spark.hadoop.google.cloud.auth.service.account.json.keyfile','/mnt/secrets/sarc-bucket-sa.json'), \
                                   ('spark.hadoop.fs.gs.project.id','sarcasm-3wx3ce6drvftuy'), \
                                   ('spark.hadoop.fs.gs.system.bucket','sarc-bucket-3wx3ce6drvftuy')]



In [ ]:
def start_spark(spark_conf, driver_ip):
    
    conf = pyspark.SparkConf().setAll(spark_conf)
    spark = SparkSession.builder.master("k8s://https://kubernetes.default.svc.cluster.local:443")\
    .appName("sarc").config(conf=conf).getOrCreate()
    sc = spark.sparkContext
    
    return sc
    

In [ ]:
def load_data(dataset):
    
    sarc = spark.read.csv("gs://sarc-bucket-3wx3ce6drvftuy/{}.csv".format(dataset), 
                          inferSchema=True, header=False, sep = '\t')
    
    sarcastic = sarc.where(col('label')==1)
    non_sarcastic = sarc.where(col('label')==0)
    
    return sarcastic, non_sarcastic

sarcastic, non_sarcastic = load_data(dataset)

In [35]:
def createTokenizer(currentDir, bertDir, vocabDir):
    
    token_dict = {}
    with codecs.open(vocabDir, 'r', 'utf8') as reader:
        for line in reader:
            token = line.strip()
            token_dict[token] = len(token_dict)

    tokenizer = Tokenizer(token_dict, cased=True)
    return tokenizer

tokenizer = createTokenizer(currentDir, bertDir, vocabDir)

In [ ]:
def generate_epochs(sarcastic, non_sarcastic, n_epochs, seed=1):
    
    
    
    

In [12]:
# params for training

SEQ_LEN = 128
BATCH_SIZE = 20
EPOCHS = 7
LR = 1e-1

# loading in BERT

bert = load_trained_model_from_checkpoint(config_path,
                                         checkpoint_path,
                                         training=True,
                                         trainable=True,
                                         seq_len=SEQ_LEN)

TypeError: 'method' object is not subscriptable